<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li></ul></div>

# Description

We want to: 
- build a good set of functions to compute the size control
- run a sensitivity analysis of the size control as a function of the mass scaling and the relative degradation rate

The parameters that we expect to have a significant impact are: 
- the type of transition
- the rate of transition `k`
- the degradation ratio `epsilon`

The way we do it is: 

In a script:
- Run simulations for a single cell (I don't think it's useful to use populations in this case)
- Save simulation results
- Compute slopes for the different metrics we plot under `plot_size_control`

Here
- Analyze the results

# Load

In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from RBmodel import cell_models

In [4]:
from RBmodel import load_utils, plot_utils, analysis

In [18]:
import pickle

In [28]:
from sklearn.linear_model import LinearRegression

In [5]:
params = cell_models.DEFAULT_PARAMS.copy()

In [64]:
1 != 2

True